In [19]:
# Import libraries
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
import numpy as np
import re
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import warnings
import random
warnings.filterwarnings("ignore")

In [6]:
# Import csv file
cities_path = "../data/hotels_items.csv"

In [8]:
# Create dataframe
cities_df = pd.read_csv(cities_path, usecols = ["city"])

In [9]:
cities_df.head()

,city
0,"Mutterstadt, Germany"
1,"Passau, Germany"
2,"Hamburg, Germany"
3,"Stralsund, Germany"
4,"Wolfsburg, Germany"


In [10]:
# Sanity check
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 727491 entries, 0 to 727490
Data columns (total 1 columns):
city    727491 non-null object
dtypes: object(1)
memory usage: 5.6+ MB


In [11]:
cities_df.drop_duplicates(inplace = True)

In [12]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27194 entries, 0 to 727408
Data columns (total 1 columns):
city    27194 non-null object
dtypes: object(1)
memory usage: 424.9+ KB


In [13]:
city_list = cities_df["city"].tolist()

In [14]:
# # Create empty columns for the values that will be pulled through API
hotels_df = pd.DataFrame()
hotels_df["rating"] = np.nan
hotels_df["name"] = None
hotels_df["address"] = None
hotels_df["locality"] = None
hotels_df["landmarks"] = None
hotels_df["price"] = np.nan

In [15]:
# List lenght
len(city_list)

27194

In [121]:
options = Options()
options.headless = True
driver = webdriver.Chrome('/usr/bin/chromedriver', chrome_options=options)
url = "https://www.hotels.com/"

#response.implicitly_wait(10)

for city in city_list:
    print('Processing: ' + city)

    searchKey = city
    checkInDate = '08/27/2020' #Format %m/%d/%Y
    checkOutDate = '08/29/2020' #Format %m/%d/%Y

    driver.get(url)

    searchKeyElement = driver.find_elements_by_name("q-destination")
    checkInElement = driver.find_elements_by_name("q-localised-check-in")
    checkOutElement = driver.find_elements_by_name("q-localised-check-out")
    submitButton = driver.find_elements_by_css_selector("button.cta-strong")
    closeIconForPopup = driver.find_elements_by_css_selector("span.close-icon")
    h1QueryHeading = driver.find_elements_by_css_selector("h1.widget-query-heading")

    
    if searchKeyElement and checkInElement and checkOutElement and closeIconForPopup and navBar:
        print('All search elements found.')
        searchKeyElement[0].clear()
        searchKeyElement[0].send_keys(searchKey)
        checkInElement[0].clear()
        checkInElement[0].send_keys(checkInDate)
        checkOutElement[0].clear()
        checkOutElement[0].send_keys(checkOutDate)
        closeIconForPopup[0].click()
        # Click somewhere to get rid of the autocomplete box
        h1QueryHeading[0].click()
        time.sleep(1)
        submitButton[0].click()
        print('Pressed the submit button.')
        time.sleep(1)
        
        print('Scrolling a few times.')
        for i in range(3):
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.5)

        hotels = driver.find_elements_by_css_selector('section.hotel-wrap')

        for index, hotel in enumerate(hotels):
            hotel_name = hotel.find_element_by_css_selector('a.property-name-link').text.strip()
            print(hotel_name)
            try:
                hotel_rating = float(hotel.find_element_by_css_selector('div.ta-logo').text.split()[-1])
                print(hotel_rating)
            except:
                print("Rating not available")
            hotel_address = hotel.find_element_by_css_selector('span.address').text
            print(hotel_address)
            hotel_locality = hotel.find_element_by_css_selector("a.map-link").text.strip()
            print(hotel_locality)
            hotel_landmarks = hotel.find_element_by_css_selector("ul.property-landmarks").text
            print(hotel_landmarks)
            try:
                hotel_price = float(hotel.find_element_by_css_selector("a.price-link").find_element_by_tag_name("ins").text.strip("$"))
                print(hotel_price)
            except:
                print("Price not available")
#             # Fill each column of the row with the values gotten from the API response
#             hotels_df.at[index, "rating"] = hotel_rating
#             hotels_df.at[index, "name"] = hotel_name
#             hotels_df.at[index, "address"] = hotel_address
#             hotels_df.at[index, "locality"] = hotel_locality
#             hotels_df.at[index, "landmarks"] = hotel_landmarks
#             hotels_df.at[index, "price"] = hotel_price
driver.quit()

Processing: Mutterstadt, Germany
All search elements found.
Pressed the submit button.
Scrolling a few times.
RheinCity Hotel
3.5
Zollhofstraße 11, Ludwigshafen, RP, 67059, Germany
Ludwigshafen
5.4 miles to Mutterstadt
3.3 miles to Mannheim (MHG)
107.0
NYX Hotel Mannheim by Leonardo Hotels
4.0
F4, 4-11, Mannheim, BW, 68159, Germany
Mannheim
6.2 miles to Mutterstadt
2.8 miles to Mannheim (MHG)
78.0
Radisson Blu Hotel, Mannheim
4.5
Quadrant Q7, 27, Mannheim, 68161, Germany
Mannheim
6.4 miles to Mutterstadt
2.3 miles to Mannheim (MHG)
Price not available
Leonardo Royal Hotel Mannheim
3.5
Augustaanlage 4-8, Mannheim, BW, 68165, Germany
Mannheim
6.5 miles to Mutterstadt
2.0 miles to Mannheim (MHG)
80.0
Leonardo Hotel Mannheim City Center
3.5
N6, 3, Mannheim, BW, 68161, Germany
Mannheim
6.2 miles to Mutterstadt
2.4 miles to Mannheim (MHG)
71.0
ACHAT Comfort Frankenthal/Pfalz
3.5
Mahlastrasse 18, Frankenthal, RP, 67227, Germany
Frankenthal
5.9 miles to Mutterstadt
8.3 miles to Mannheim (MHG)


KeyboardInterrupt: 

In [122]:
driver.quit()